# Self-translate

英語以外の言語のプロンプトでは言語モデルはその能力をフルに活用できていない，という研究に関する論文．self-translateは入力を一度英語に翻訳して出力を得てから元の言語に戻す手法．
- 論文:https://arxiv.org/abs/2308.01223

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/develop/nlp/llm_prompt_engineering/self_translate.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install openai

In [3]:
import os
from openai import OpenAI
client = OpenAI()

#os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

### 通常のプロンプト(Few-shot prompting)

In [1]:
prompt = """
アンディはゼラニウムを90本植え、ペチュニアをゼラニウムより40本少なく植える全部で何本の花を植えたでしょう？
"""

In [4]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt}]
    )

print(response.choices[0].message.content)

アンディはゼラニウムを90本植えたので、ペチュニアは90本 - 40本 = 50本植えた。
全部でアンディは90本のゼラニウムと50本のペチュニアを植えたので、合計で90本 + 50本 = 140本の花を植えた。


### Self-translate

In [8]:
# 英語に翻訳する
prompt1 = """
Japanese:メアリは緑の魔女を叩かなかった．
English:Mary did not slap the green witch.

Japanese:アンディはゼラニウムを90本植え、ペチュニアをゼラニウムより40本少なく植える。
全部で何本の花を植えたでしょう？
English:
"""


In [9]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt1}]
    )
    
output1 = response.choices[0].message.content

print(response.choices[0].message.content)

Andy planted 90 geraniums and planted 40 fewer petunias than geraniums.
How many flowers did he plant in total?


In [10]:
# 問題を解く
prompt2 = """
Question: John had 12 apples and ate a quarter of them. How many apples does he have now?
Stepwise solution: John ate 12÷4=3 apples. Therefore, he has 12-3=9 apples now. The answer is 9.

Question: {output1_}
Stepwise solution:
""".format(output1_ = output1)

In [11]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt2}]
    )

output2 = response.choices[0].message.content

print(response.choices[0].message.content)

Andy planted 90-40=50 petunias.
Therefore, he planted a total of 90+50=140 flowers.


In [12]:
# 元の言語に直す
prompt3 = """
English:Mary did not slap the green witch.
Japanese:メアリは緑の魔女を叩かなかった．

English:{output2_}
Japanese:
""".format(output2_ = output2)

In [13]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": prompt3}]
    )

print(response.choices[0].message.content)

アンディはペチュニアを90-40=50株植えました。
したがって、彼は合計で90+50=140本の花を植えました。
